In [3]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

True

In [8]:
pc = Pinecone(api_key=os.getenv('PINECONE_API'))
pc.create_index(
    name="rate-my-professor-rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud='aws', region='us-east-1'),

)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '3ac746557c629cd9a2b57ab30020fb4b', 'Date': 'Tue, 27 Aug 2024 14:58:57 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [1]:
import json
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Great professor, explains concepts clearly.'},
 {'professor': 'Dr. Jane Doe',
  'subject': 'Advanced Algorithms',
  'stars': 5,
  'review': 'Challenging course but very rewarding.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Data Structures',
  'stars': 4,
  'review': 'Very knowledgeable but sometimes hard to follow.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Operating Systems',
  'stars': 5,
  'review': 'Engaging lectures and helpful office hours.'},
 {'professor': 'Dr. Sarah Davis',
  'subject': 'Database Systems',
  'stars': 4,
  'review': 'Good course material and fair exams.'},
 {'professor': 'Dr. Robert Wilson',
  'subject': 'Computer Networks',
  'stars': 3,
  'review': 'Interesting content but lectures can be dry.'},
 {'professor': 'Dr. Laura Martinez',
  'subject': 'Software Engineering',
  'stars': 5,
  'review': 'Very practical and hands-on approach.

In [10]:
process_data = []
client = OpenAI(api_key=os.getenv('OPEN_AI_API'))

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    process_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
        } 
    )

In [ ]:
index = pc.Index('rate-my-professor-rag')
index.upsert(
    vectors=process_data,
    namespace='ns1',
)

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}